In [1]:
# 🍔 AI Food Calorie Estimator with Gemini 2.0 Flash API
# -------------------------------------------------------
# Features:
#  1. Upload a food image
#  2. Detect if it's a food item
#  3. If not food -> show error
#  4. If food -> analyze and give estimated calories & details

# -------------------- 1. INSTALL LIBRARIES --------------------
!pip install --quiet gradio pillow requests

import gradio as gr
import requests
import base64
from PIL import Image
import io

# -------------------- 2. GEMINI API CONFIG --------------------
GEMINI_API_KEY = "API_KEY"
GEMINI_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

# -------------------- 3. FUNCTION TO ANALYZE FOOD IMAGE --------------------
def analyze_food_image(img: Image.Image):
    # Convert image to base64 for API
    buffered = io.BytesIO()
    img.convert("RGB").save(buffered, format="JPEG")
    img_base64 = base64.b64encode(buffered.getvalue()).decode()

    # Prompt for Gemini
    prompt = """
    You are a professional food nutrition expert.
    Task:
    1. Identify if the uploaded image contains food.
    2. If it is NOT a food item, return exactly: "NOT_FOOD".
    3. If it IS a food item:
       - Provide the food name.
       - Estimate total calories (approximate).
       - Provide brief nutritional details (like protein, carbs, fats).
       - Suggest if it is healthy or should be eaten in moderation.
    4. Return the answer in a clean and easy-to-read format for a class project.
    Example format:
    🍕 Food: Pizza
    🔹 Estimated Calories: ~285 kcal per slice
    🔹 Nutritional Info: High in carbs and fats
    🔹 Health Tip: Enjoy in moderation due to high calories
    """

    payload = {
        "contents": [
            {
                "parts": [
                    {"text": prompt},
                    {
                        "inline_data": {
                            "mime_type": "image/jpeg",
                            "data": img_base64
                        }
                    }
                ]
            }
        ]
    }

    headers = {
        "Content-Type": "application/json",
        "X-goog-api-key": GEMINI_API_KEY
    }

    try:
        response = requests.post(GEMINI_URL, headers=headers, json=payload)
        result = response.json()

        # Extract text safely
        try:
            output_text = result["candidates"][0]["content"]["parts"][0]["text"].strip()
        except:
            return "❌ Error: Could not analyze the image. Please try again."

        # Handle NOT_FOOD
        if "NOT_FOOD" in output_text.upper():
            return "❌ Error: Uploaded image does not appear to be food. Please upload a clear food image."

        return f"🍽 Food Analysis Result:\n\n{output_text}"

    except Exception as e:
        return f"❌ API Error: {str(e)}"

# -------------------- 4. GRADIO WEB APP --------------------
demo = gr.Interface(
    fn=analyze_food_image,
    inputs=gr.Image(type="pil", label="Upload Food Image"),
    outputs=gr.Textbox(label="Result"),
    title="🍔 AI Food Calorie Estimator (Gemini Powered)",
    description="Upload a food image. If it's valid, you'll get the estimated calories and health tips. If not, you'll get an error."
)

# -------------------- 5. LAUNCH APP --------------------
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dc16a984cacb88312d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
